In [1]:
import urllib
import bs4
import re
import pandas as pd
import random
#
from time import sleep

In [2]:
url_landing = "https://leagueoflegends.fandom.com/wiki/List_of_champions/Base_statistics"
req = urllib.request.Request(url_landing, headers={'User-Agent':'Mozilla/5.0'})
webpage = urllib.request.urlopen(req).read()
stats_champions_lol = bs4.BeautifulSoup(webpage, "lxml")

###

url_landing = "https://leagueoflegends.fandom.com/wiki/List_of_champions_in_Dawn_of_Heroes_(Teamfight_Tactics)"
req = urllib.request.Request(url_landing, headers={'User-Agent':'Mozilla/5.0'})
webpage = urllib.request.urlopen(req).read()
list_champs_tft = bs4.BeautifulSoup(webpage, "lxml")

In [3]:
headers = []
for item in stats_champions_lol.findAll('th')[0:19]:
    #print(item.getText())
    headers.append(item.getText())
    
champions_stats = []

for item in stats_champions_lol.findAll('td'):
    #print(item.getText())
    champions_stats.append(item.getText())
    
n = len(headers)
lol_champ_w_stats = [champions_stats[i * n:(i + 1) * n] for i in range((len(champions_stats) + n - 1) // n )]
lol_champ_w_stats = lol_champ_w_stats[:-1]

df_lol_champs = pd.DataFrame(lol_champ_w_stats, columns = headers)

In [4]:
df_lol_champs['Champions'][130]

' Tristana'

In [5]:
#get headers

url_champ = "https://leagueoflegends.fandom.com/wiki/Aatrox/LoL"
req = urllib.request.Request(url_champ, headers={'User-Agent':'Mozilla/5.0'})
page_champ = bs4.BeautifulSoup(urllib.request.urlopen(req).read(), "lxml")

headers_stats = ['Title']
spells = ['Passive','Q Spell','W Spell','E Spell','Ultimate']
kda = ['KDA_kills','KDA_deaths','KDA_assists']

for item in page_champ.findAll('h3',{'class':'pi-data-label pi-secondary-font'})[:11]:
    headers_stats.append(item.getText())

headers_stats = headers_stats + spells

url_champ = "https://www.leagueofgraphs.com/champions/stats/ezreal"
req = urllib.request.Request(url_champ, headers={'User-Agent':'Mozilla/5.0'})
page_champ = bs4.BeautifulSoup(urllib.request.urlopen(req).read(), "lxml")

for item in page_champ.findAll('div',{'class':'pie-chart-title pie-chart-title-bottom'}):
    headers_stats.append(item.getText().replace(' ','').replace('\n','').replace('\r',''))
    
for item in page_champ.findAll('div',{'class':'title'})[2:]:
    headers_stats.append(item.getText().replace(' ','').replace('\n','').replace('\r',''))

headers_stats = headers_stats + kda

#init
i = 1
items_champ = []

#get data

for champ in df_lol_champs['Champions']:       
    
    #get data from fandom
    url_champ = "https://leagueoflegends.fandom.com/wiki/"+champ.replace(' ','_')+"/LoL"
    req = urllib.request.Request(url_champ, headers={'User-Agent':'Mozilla/5.0'})
    page_champ = bs4.BeautifulSoup(urllib.request.urlopen(req).read(), "lxml")
    
    for item in page_champ.findAll('div',{'class':'pi-data-value pi-font'})[:12]:
        items_champ.append(item.getText())
        
    for item in page_champ.findAll('span',{'class':'mw-headline'})[1:6]:
        items_champ.append(item.getText())
    
    
    #get data from league of graphs
    
    url_champ = "https://www.leagueofgraphs.com/champions/stats/"+champ.replace(' ','').replace("'","").replace('.','').replace('Mega','').replace('Wukong','monkeyking').lower()
    req = urllib.request.Request(url_champ, headers={'User-Agent':'Mozilla/5.0'})
    page_champ = bs4.BeautifulSoup(urllib.request.urlopen(req).read(), "lxml")
    
    for item in page_champ.findAll('div',{'class':'pie-chart'}):
        items_champ.append(item.getText().replace(' ','').replace('\n','').replace('\r',''))
    
    for item in page_champ.findAll('div',{'class':'number solo-number'})[1:]:
        items_champ.append(item.getText().replace(' ','').replace('\n','').replace('\r',''))

    for item in page_champ.findAll('span',{'class':'kills'})[0]:
        items_champ.append(item.replace(' ','').replace('\n','').replace('\r',''))
    for item in page_champ.findAll('span',{'class':'deaths'})[0]:
        items_champ.append(item.replace(' ','').replace('\n','').replace('\r',''))
    for item in page_champ.findAll('span',{'class':'assists'})[0]:
        items_champ.append(item.replace(' ','').replace('\n','').replace('\r',''))
    
            
    print(str(i)+" / "+str(len(df_lol_champs['Champions']))+" champions importés")
    
    i+=1
    
    sleep(random.randint(5,10))

###    
    


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)>

In [6]:
n = len(headers_stats)
l_test = [items_champ[i * n:(i + 1) * n] for i in range((len(items_champ) + n - 1) // n )]

test = pd.DataFrame(l_test, columns = headers_stats)

NameError: name 'items_champ' is not defined

In [7]:
df_lol_champs = pd.concat([df_lol_champs, test.reindex(df_lol_champs.index)], axis=1)

NameError: name 'test' is not defined

In [9]:
df_lol_champs.head()

,Champions,HP,HP+,HP5,HP5+,MP,MP+,MP5,MP5+,AD,AD+,AS,AS+,AR,AR+,MR,MR+,MS,Range
0,Aatrox,650,+114,3,+1,0,+0,0,+0,60,+5,0.651,+2.5%,38,+4.45,32,+2.05,345,175
1,Ahri,570,+96,2.5,+0.6,418,+25,8,+0.8,53,+3,0.668,+2%,18,+4.7,30,+1.3,330,550
2,Akali,570,+119,9,+0.9,200,+0,50,+0,62,+3.3,0.625,+3.2%,23,+4.7,37,+2.05,345,125
3,Akshan,630,+104,3.75,+0.65,350,+40,8.175,+0.7,52,+3.5,0.638,+4%,26,+4.2,30,+1.3,330,500
4,Alistar,670,+120,8.5,+0.85,350,+40,8.5,+0.8,62,+3.75,0.625,+2.125%,44,+4.7,32,+2.05,330,125


In [8]:
df_lol_champs.tail()

,Champions,HP,HP+,HP5,HP5+,MP,MP+,MP5,MP5+,AD,AD+,AS,AS+,AR,AR+,MR,MR+,MS,Range
158,Zeri,600,+109,3.25,+0.7,250,+45,6,+0.8,50,+1.5,0.658,+2%,20,+4.2,30,+1.3,325,500
159,Ziggs,606,+106,6.5,+0.6,480,+23.5,8,+0.8,54,+3.1,0.656,+2%,22,+4.5,30,+1.3,325,550
160,Zilean,574,+96,5.5,+0.5,452,+50,11.34,+0.8,52,+3,0.625,+2.13%,24,+5,30,+1.3,335,550
161,Zoe,630,+106,6.5,+0.6,425,+25,8,+0.65,58,+3.3,0.625,+2.5%,21,+4.7,30,+1.3,340,550
162,Zyra,574,+93,5.5,+0.5,418,+25,13,+0.4,53,+3.2,0.625,+2.11%,29,+4.2,30,+1.3,340,575


In [10]:
df_lol_champs.to_csv('export_lol_champs.csv')